In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
%%capture
!pip install hazm transformers torch nltk -q

In [2]:
import json
import logging
import re
import string
from collections import defaultdict

from transformers import pipeline
from nltk.tokenize import word_tokenize
import nltk
from hazm import Normalizer, Lemmatizer, word_tokenize

nltk.download('punkt')

logging.basicConfig(
    filename='/kaggle/working/qa_errors.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    encoding='utf-8'
)

# Stopwords فارسی
PERSIAN_STOPWORDS = set([
    'آیا', 'ای', 'اما', 'اگر', 'از', 'اش', 'افراد', 'است', 'اسلامی', 'اشاره', 'اصلاً',
    'اعلام', 'افزود', 'اکنون', 'البته', 'اند', 'ان', 'انجام', 'آن', 'آنان', 'آنه', 'آنها',
    'او', 'اول', 'ایم', 'اینجا', 'اینان', 'این', 'اینه', 'اینها', 'باشد', 'بار', 'باره',
    'باید', 'بالا', 'بالای', 'بر', 'برا', 'برای', 'برخی', 'برداری', 'برخوردار', 'بروز',
    'بزرگ', 'بس', 'بسیار', 'بطور', 'بعد', 'بعدا', 'بلافاصله', 'بلکه', 'بله', 'بنابراین',
    'بنده', 'به', 'بها', 'بود', 'بوده', 'بودند', 'بیرون', 'بی', 'بین', 'بیش', 'بیشتر', 'تا',
    'تمام', 'تحت', 'تر', 'تر از', 'ترک', 'تعداد', 'توسط', 'توی', 'تمامی', 'تنها', 'تو',
    'تواند', 'تومان', 'ثبت', 'ثانیاً', 'جاری', 'جدا', 'جز', 'جلوگیری', 'جمعی', 'چرا',
    'چه', 'چیز', 'چند', 'چنین', 'حتی', 'حد', 'حداکثر', 'حدود', 'حق', 'حضرت', 'حتماً',
    'حاضر', 'خود', 'خویش', 'خواه', 'خواهند', 'خوب', 'خودش', 'خیلی', 'دارای', 'داشت',
    'داشته', 'داشتند', 'در', 'درباره', 'درحال', 'درست', 'درصورتی', 'درعین', 'دریافت',
    'دسته', 'دگر', 'دهد', 'دهند', 'دو', 'دوم', 'دوباره', 'دیر', 'دیگر', 'دیگران', 'را',
    'راه', 'راجع', 'رسماً', 'رسید', 'رشد', 'روی', 'زود', 'زیاد', 'زیر', 'سابق', 'سراسر',
    'سری', 'سریعا', 'سریعتاً', 'سعی', 'سوم', 'شد', 'شده', 'شدند', 'شش', 'شما', 'شان',
    'شو', 'شود', 'شوند', 'صفر', 'صرفاً', 'صورت', 'ضد', 'ضمن', 'طبیعی', 'طور', 'طی',
    'ظرف', 'عالی', 'عبارتند', 'عدم', 'عقب', 'علت', 'علیه', 'عنوان', 'غیر', 'فبها', 'فر',
    'فقط', 'فعلاً', 'فوق', 'قابل', 'قبل', 'قبلاً', 'قدر', 'قصد', 'قطعاً', 'قرار', 'قریب',
    'گذشته', 'گذاشتن', 'گر', 'گرفت', 'گرفته', 'گروهی', 'گه', 'گاهی', 'گو', 'گفت', 'گفتند',
    'لذا', 'لطفاً', 'ما', 'مانند', 'مابقی', 'مبتنی', 'متاسفانه', 'مجدد', 'مجدداً', 'محسوب',
    'مختلف', 'مدام', 'مدتی', 'مراجعه', 'مرتب', 'مردم', 'موجب', 'مگر', 'من', 'منتهی',
    'مورد', 'میزان', 'میلیون', 'می', 'نماید', 'نمایند', 'نمی', 'ناشی', 'نخست', 'نخستین',
    'نه', 'نوعی', 'نیاز', 'نزدیک', 'نسبت', 'نشان', 'نشده', 'نظر', 'نفر', 'نقش', 'نیز',
    'نیست', 'نمی‌تواند', 'ها', 'های', 'هایش', 'هر', 'هرگز', 'هزار', 'هست', 'هستند',
    'هم', 'همان', 'همچنان', 'همچنین', 'همه', 'همواره', 'هیچ', 'و', 'وجود', 'ولی', 'وی',
    'پارسال', 'پایین', 'پایین‌تر', 'پشت', 'پس', 'پیش', 'پیشتر', 'چطور', 'چندین', 'کار',
    'کامل', 'کاملاً', 'کجا', 'کرد', 'کرده', 'کردند', 'کس', 'کسانی', 'کل', 'کلیه', 'کم',
    'کماکان', 'کنار', 'کند', 'کنند', 'که', 'کو', 'کی', 'گذشته', 'گذاشته', 'گیر', 'یک'
])

normalizer = Normalizer()
lemmatizer = Lemmatizer()

def normalize_text(text):
    if not text:
        return ""
    text = normalizer.normalize(text.lower().strip())
    text = re.sub(f'[{string.punctuation}]', ' ', text)
    text = re.sub(r'\b\d+\b', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in PERSIAN_STOPWORDS]
    return ' '.join(tokens)

def is_meaningful_answer(answer, question):
    if not answer or len(answer.strip()) < 20:
        return False
    if re.match(r'^\d+$', answer.strip()) or re.match(r'ص\s*\d+', answer.strip()):
        return False
    if len(answer.strip().split()) < 5:
        return False
    if answer.strip() in question:
        return False
    question_tokens = set(normalize_text(question).split())
    answer_tokens = set(normalize_text(answer).split())
    if not (question_tokens & answer_tokens):
        return False
    return True

def compute_answer_similarity(answer1, answer2):
    tokens1 = set(normalize_text(answer1).split())
    tokens2 = set(normalize_text(answer2).split())
    if not tokens1 or not tokens2:
        return 0.0
    return len(tokens1 & tokens2) / len(tokens1 | tokens2)

def remove_similar_answers(candidates, threshold=0.5):
    unique_candidates = []
    seen = set()
    for cand in sorted(candidates, key=lambda x: x['score'], reverse=True):
        norm_ans = normalize_text(cand['answer'])
        if not norm_ans:
            continue
        if any(compute_answer_similarity(norm_ans, sa) > threshold for sa in seen):
            continue
        seen.add(norm_ans)
        unique_candidates.append(cand)
    return unique_candidates[:3]

def select_relevant_contexts(question, contexts, top_n=3):
    question_tokens = set(normalize_text(question).split())
    scores = []
    for ctx in contexts:
        ctx_text = normalize_text(ctx['context'])
        ctx_tokens = set(ctx_text.split())
        score = len(question_tokens & ctx_tokens) / len(question_tokens | ctx_tokens)
        for keyword in question_tokens:
            if keyword in ctx_text:
                score += 0.1
        scores.append((ctx, score))
    return [ctx for ctx, _ in sorted(scores, key=lambda x: x[1], reverse=True)[:top_n]]


2025-08-02 07:18:56.631002: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754119137.001260      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754119137.114524      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:

def process_questions():
    try:
        with open('/kaggle/input/newtestd/Test_data.json', 'r', encoding='utf-8') as f:
            test_data = json.load(f)
        with open('/kaggle/input/aggregate-and-test/aggregate.json', 'r', encoding='utf-8') as f:
            contexts = json.load(f)

        qa_pipeline = pipeline(
            'question-answering',
            model='cis-lmu/glot500-base',
            tokenizer='cis-lmu/glot500-base',
            framework='pt',
            max_answer_len=100,
            min_answer_len=5,
            top_k=10,
            handle_impossible_answer=True
        )

        results = []
        for q in test_data:
            q_id = q['id']
            question = q['question']
            relevant = select_relevant_contexts(question, contexts)
            if not relevant:
                results.append({'id': q_id, 'question': question, 'top_answers': []})
                continue
            candidates = []
            for ctx in relevant:
                try:
                    answers = qa_pipeline(question=question, context=ctx['context'])
                    if not isinstance(answers, list):
                        answers = [answers]
                    for ans in answers:
                        if is_meaningful_answer(ans['answer'], question):
                            candidates.append({
                                'answer': ans['answer'].strip(),
                                'score': ans['score'],
                                'context': ctx['context']
                            })
                except Exception as e:
                    logging.error(f"Error: {str(e)}")
            unique = remove_similar_answers(candidates)
            results.append({'id': q_id, 'question': question, 'top_answers': unique})

        with open('/kaggle/working/qa_results.json', 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)

    except Exception as e:
        logging.error(f"Error in process_questions: {str(e)}")
        raise

if __name__ == "__main__":
    process_questions()


config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at cis-lmu/glot500-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/496 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/7.66M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [4]:

import json

with open('/kaggle/working/qa_results.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

print(json.dumps(data, indent=4, ensure_ascii=False))


[
    {
        "id": 1,
        "question": "به موجب  پیمان سنت پترزبورگ چه قسمت هایی از ایران به روسیه واگذار شدند؟",
        "top_answers": [
            {
                "answer": "شد. این پیمان به موجب آن ایران مجبور شد سرزمین‌های وسیعی در قفقاز شمالی، قفقاز جنوبی و آذربایجان شامل دربند (داغستان)، بادکوبه و اراضی تابع",
                "score": 2.4586455765529536e-05,
                "context": "پیمان سنت پترزبورگ در تاریخ ۲۳ سپتامبر ۱۷۲۳ (۱۲ سپتامبر به تقویم ژولینی) بین ایران و روسیه امضا شد. این پیمان به موجب آن ایران مجبور شد سرزمین‌های وسیعی در قفقاز شمالی، قفقاز جنوبی و آذربایجان شامل دربند (داغستان)، بادکوبه و اراضی تابع اطراف آنها را به روسیه واگذار کند. همچنین تمام نواحی شمال ایران شامل استان‌های گیلان، مازندران و استرآباد نیز تحت این پیمان به روسیه تعلق گرفت. این قرارداد پس از حمله پتر یکم به ایران و به منظور تثبیت اوضاع داخلی امضا شد. طبق این پیمان، سربازان روسی به شاه تهماسب دوم کمک کردند تا آرامش و صلح داخلی را برقرار کند. اهمیت تاریخی این پیمان در آن است که به عنوان 